In [3]:
from torchvision import datasets
import torch
import dlc_practical_prologue as prologue

# Import all models
from BaseNet import *
from ConvNet1 import *

mini_batch_size = 1000
nb_epochs = 300
nb_runs = 10
eta = 0.001

train_input, train_target, train_classes, test_input, test_target, test_classes = \
    prologue.generate_pair_sets(nb=1000)

In [16]:
from importlib import reload
import _1channel2images
reload(_1channel2images)
from _1channel2images import *

print("Working with 1channel2images framework, nb_classes = ", nb_classes)

#model = BaseNet1C(nb_classes)
model_1C = ConvNet1_1C(nb_classes)
optimizer_1C = torch.optim.SGD(model_1C.parameters(), lr=eta, momentum=0.95)
test_results_1C = multiple_training_runs(model_1C, 2, optimizer_1C, train_input, train_classes,
                                      test_input, test_target, test_classes, mini_batch_size, nb_epochs)
write_to_csv('1channel2images.csv', model_1C, test_results_1C)

Working with 1channel2images framework, nb_classes =  10


KeyboardInterrupt: 

In [22]:
# This is just for visualization of our results but it will have to be taken away for the report 
# since we can't use any additional libraries
import pandas as pd
from importlib import reload
import _1channel2images
reload(_1channel2images)
from _1channel2images import *

print(test_results_1C)
write_to_csv('1channel2images.csv', model_1C, test_results_1C)
data = pd.read_csv('1channel2images.csv')
data.head()

(167.95791387557983, 10.196516590709884, 0.8707499999999999, 0.0017677669529664096, 0.912, 0.004242640687119289)
Overwriting file


,Model,Number of parameters,Training time,Mean digits accuracy (test set),Std digits accuracy,Mean accuracy (test set),Std accuracy
0,ConvNet1_1C,72226,167.96,0.8707,0.0018,0.912,0.0042


In [5]:
class NetSharing1(nn.Module):
    def __init__(self, nb_hidden=100):
        super(NetSharing1, self).__init__()
        self.name = "NetSharing1"
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(512, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, 2)
 
        self.criterion = nn.CrossEntropyLoss()
        self.target_type = torch.LongTensor
 
    def forward(self, x):
        x_0 = x[:,0,:,:].view(-1,1,14,14)
        x_1 = x[:,1,:,:].view(-1,1,14,14)
        x1 = F.relu(F.max_pool2d(self.conv1(x_0), kernel_size=2, stride=2))
        x1 = F.relu(F.max_pool2d(self.conv2(x1), kernel_size=2, stride=2))
        x1 = x1.view(-1, 256)
 
        x2 = F.relu(F.max_pool2d(self.conv1(x_1), kernel_size=2, stride=2))
        x2 = F.relu(F.max_pool2d(self.conv2(x2), kernel_size=2, stride=2))
        x2 = x2.view(-1, 256)
 
        x = torch.cat((x1,x2),1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [21]:
from importlib import reload
import weight_sharing
reload(weight_sharing)
from weight_sharing import *

print("Working with weight_sharing framework")

#model_2C = ConvNet1_2C(nb_classes)
model_ws = NetSharing1()
optimizer_ws = torch.optim.SGD(model_ws.parameters(), lr=0.00001, momentum=0.95)  
test_results_ws = multiple_training_runs(model_ws, 2, optimizer_ws, train_input, train_target,
                           test_input, test_target, test_classes, mini_batch_size, nb_epochs)
write_to_csv('weightsharing.csv', model_ws, test_results_ws)

Working with weight_sharing framework
phase: train, epoch: 100, loss: 0.52017, acc: 0.7660
phase: val, epoch: 100, loss: 0.52329, acc: 0.7630
phase: train, epoch: 200, loss: 0.40882, acc: 0.8180
phase: val, epoch: 200, loss: 0.40815, acc: 0.8200
phase: train, epoch: 300, loss: 0.35544, acc: 0.8430
phase: val, epoch: 300, loss: 0.35500, acc: 0.8440
Training complete in 2 min 48 s
Best val acc: 0.8440
phase: train, epoch: 100, loss: 0.50940, acc: 0.7710
phase: val, epoch: 100, loss: 0.51263, acc: 0.7670
phase: train, epoch: 200, loss: 0.39975, acc: 0.8250
phase: val, epoch: 200, loss: 0.39909, acc: 0.8240
phase: train, epoch: 300, loss: 0.34771, acc: 0.8500
phase: val, epoch: 300, loss: 0.34729, acc: 0.8500
Training complete in 2 min 43 s
Best val acc: 0.8500


FileNotFoundError: [Errno 2] No such file or directory: 'weightsharing.csv'

In [25]:
from importlib import reload
import weight_sharing
reload(weight_sharing)
from weight_sharing import *
import pandas as pd 

print(test_results_ws)
write_to_csv('weightsharing.csv', model_ws, test_results_ws)
data = pd.read_csv('weightsharing.csv')
data.head()

(165.5573890209198, 3.983937261813232, 0.7695000000000001, 0.0007071067811865482)


,Model,Number of parameters,Training time,Mean accuracy (test set),Std accuracy
0,NetSharing1,70318,165.56,0.7695,0.0007
